In [3]:
import re
import math
import fitz
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import es_core_news_sm
from nltk.tokenize import sent_tokenize
import os
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import es_core_news_sm
import itertools
from nltk.stem import SnowballStemmer
import textacy
import regex
import unidecode
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/fasttext-sbwc.3.6.e20.vec'

In [4]:
cantidad = 100000

model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [5]:
file= '/home/erwin/Genoma/cv-parser/resumes/prueba_genoma_experiencia.pdf'
#file ='/home/erwin/Genoma/cv-parser/resumes/1574824594718-Ing._Comercial_Paula_Fetis_CV.pdf'
#file ='/home/erwin/Genoma/cv-parser/resumes/1580248761709-CV_Sergio_Soto_Valdes.pdf'

In [6]:
def extract_text(path):
    '''
    Input: ruta hacia los archivos
    Salida: Texto plano como string
    '''
    with fitz.open(path) as doc:
        text = ""
        for page in doc:
            text += page.getText()
        # eliminar estos simbolos
        
        text_clean = text
        text_2 = ''
        for line in text_clean.splitlines():
            if not line.strip(): #si la linea esta vacia, saltar
                continue
            line_2=''
            for word in line.split():
                if word.isupper():
                    line_2 += word.capitalize()+' '

                else:
                    line_2 += word+ ' '
            text_2 += line_2 +'\n'
        
        simbolos = ' ,\n./:@'
        text_clean = ''
        for char in text_2:
            if (char.isalnum())| (char in simbolos):
                text_clean += char
        #print(text_clean)
        # Limpiar palabras completamente en mayusculas, es importante
        # para reconocer los nombres
        
        text = text_clean
    return text

In [7]:
text = extract_text(file)
print(text)

Curriculum Vitae 
Erwin Nicolás Paillacán Huaitro 
Enero 7, 1997 
erwinpaillacan@gmail.com 
 
569 86634232 
 
Gran Avenida 3806 
 
Región Metropolitana 
https://www.linkedin.com/in/erwinpaillacan/ 
Descripción 
Oriundo de la ciudad de Osorno, actualmente curso el 
sexto año de Ingeniería Civil Eléctrica con énfasis en 
Telecomunicaciones e Inteligencia Computacional. Me 
gusta programar, mirar series y aprender cosas intere 
santes en Coursera. Aparte de ser un apasionado por 
el mundo teleco, me gusta mucho el mundo de los da 
tos y el procesamiento de imágenes. En esto último ten 
go algunos proyectos para la Universidad en mi GitHub 
https: // github. com/ erwinpaillacan . Fecha de ti 
tulación estimada 20211. 
Antecedentes Personales 
Fecha de Nacimiento: 
7 de Enero del 1997 
Edad: 
23 años 
Nacionalidad: 
Chileno 
Cédula de Identidad: 
19.270.6769 
Experiencia 
Telecsa 
Osorno, Región de los Lagos, Chile 
Alumno en Práctica I 
Enero 19  Febrero 19 
Apoyo en área de transmisión y 

In [8]:
def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [9]:
frase_1 = 'software developer'
frase_2 = 'web developer'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.8897964


In [10]:
model.most_similar(positive=['experiencia'],topn=10)

[('experiencias', 0.7211438417434692),
 ('pericia', 0.6080396175384521),
 ('aprendidas', 0.5985227227210999),
 ('vivencia', 0.595109224319458),
 ('conocimientos', 0.5840999484062195),
 ('enriquecedora', 0.5687903761863708),
 ('adquirida', 0.5659759044647217),
 ('inexperiencia', 0.5602041482925415),
 ('aprendido', 0.5534920692443848),
 ('vivida', 0.5521409511566162)]

In [86]:
prc_description ='''ventas marketing excel'''

In [87]:
# https://github.com/prateekguptaiiitk/Resume_Filtering/blob/develop/Scoring/CV_ranking.ipynb
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        print(similar_words[i], word_value[similar_words[i]])
        print('------------------------------------------------')

ventas 1
------------------------------------------------
minoristas 0.6644167900085449
------------------------------------------------
vendidas 0.6640223264694214
------------------------------------------------
marketing 1
------------------------------------------------
márketing 0.8219829797744751
------------------------------------------------
mercadotecnia 0.7757836580276489
------------------------------------------------
excel 1
------------------------------------------------
powerpoint 0.6345570087432861
------------------------------------------------
word 0.6285042762756348
------------------------------------------------


In [88]:
# cargar los los skills de los csv

import os, json
import pandas as pd

path_to_json = 'resumes_output/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)  # for me this prints ['foo.json']
jsons = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        jsons.append(json.load(json_file))
        #print(json_text)

In [89]:
word_value.keys()

dict_keys(['ventas', 'minoristas', 'vendidas', 'marketing', 'márketing', 'mercadotecnia', 'excel', 'powerpoint', 'word'])

In [91]:
jsons[0]['Skills']

['marketing', 'ventas', 'excel', 'sap', 'office']

In [100]:
count = {}
for word in word_value.keys(): 
    count[word] = 0
    if word in jsons[i]['Skills']:
        print(word)
        count[word] += 1

ventas
marketing
excel


In [101]:
count

{'ventas': 1,
 'minoristas': 0,
 'vendidas': 0,
 'marketing': 1,
 'márketing': 0,
 'mercadotecnia': 0,
 'excel': 1,
 'powerpoint': 0,
 'word': 0}

In [102]:
### ahora veamos si resulta el ranking


no_of_cv = len(jsons)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        jsons[i]['Skills'] = [x.lower() for x in jsons[i]['Skills']]
        try:
            #if word in cvs.loc(0)['skill'][i].split() or word in cvs.loc(0)['exp'][i].split():
            if word in jsons[i]['Skills']:
                print('entre')
                count[word] += 1
        except:
            pass
    if (count[word] == 0):
        count[word] = 1
    idf[word] = math.log(no_of_cv/count[word])
print(count)
print(idf)

entre
entre
entre
{'ventas': 1, 'minoristas': 1, 'vendidas': 1, 'marketing': 1, 'márketing': 1, 'mercadotecnia': 1, 'excel': 1, 'powerpoint': 1, 'word': 1}
{'ventas': 0.0, 'minoristas': 0.0, 'vendidas': 0.0, 'marketing': 0.0, 'márketing': 0.0, 'mercadotecnia': 0.0, 'excel': 0.0, 'powerpoint': 0.0, 'word': 0.0}


In [103]:

score = {}
for i in range(no_of_cv):
    score[i] = 0
    try:
        for word in word_value.keys():
            tf = jsons[i]['Skills'].count(word) #+ cvs.loc(0)['exp'][i].split().count(word)
            score[i] += word_value[word]*tf*idf[word]
    except:
        pass

In [104]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], jsons[i]['Nombre archivo']))
    
sorted_list.sort(reverse = True)

#for s, i in sorted_list:
#    if list(cvs)[i] != '.DS_Store':
#        print(list(cvs)[i], ':', s)

In [105]:
sorted_list

[(0.0, '1580573732905-DiegoAlbertoGutierrezBejarano')]